# Логистическая регрессия. Практическая работа

## Цель практической работы

Вы уже делали задание, в котором с помощью метода ближайших соседей оценивали склонность клиента банка откликнуться или не откликнуться на предложение.

В этом практическом задании ваши цели:
*  решить эту же задачу с помощью логистической регрессии;
*  потренироваться в подборе порога; 
*  потренироваться в подборе гиперпараметров модели.

## Что входит в работу

*  Загрузить данные для задачи.
*  Обучить метод ближайших соседей с заданным количеством соседей k, вычислить метрики.
*  Обучить логистическую регрессию с параметрами по умолчанию, вычислить метрики.
*  Подобрать порог модели, вычислить метрики.
*  Подобрать гиперпараметр С (константа регуляризации) модели, вычислить метрики.

## Что оценивается

*  Выполнены все этапы задания: код запускается, отрабатывает без ошибок; подробно и обоснованно написаны текстовые выводы, где это требуется.

## Формат сдачи
Выполните предложенные задания — впишите свой код (или, если требуется, текст) в ячейки после комментариев. 

*Комментарии — это текст, который начинается с символа #. Например: # ваш код здесь.*

Сохраните изменения, используя опцию Save and Checkpoint из вкладки меню File или кнопку Save and Checkpoint на панели инструментов. Итоговый файл в формате .ipynb (файл Jupyter Notebook) загрузите в личный кабинет и отправьте на проверку.

In [1]:
# подключим библиотеки
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
# считаем данные
data = pd.read_csv('8.8 ClientsData.csv')

In [3]:
data.head()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
0,49,1,0,1,2,1,5000.0,1,1,2,0
1,32,1,0,1,3,3,12000.0,1,1,1,0
2,52,1,0,1,4,0,9000.0,2,1,0,0
3,39,1,0,1,1,1,25000.0,1,1,3,0
4,30,1,0,0,0,0,12000.0,2,1,2,0


В этом ноутбуке нам придётся подбирать гиперпараметры модели, а ещё порог. Поэтому, чтобы не переобучиться, разобъём данные на трейн, валидацию и тест.

*  Обучать модели будем на тренировочных данных.
*  Подбирать необходимые величины — по валидации.
*  Оценивать качество — на тесте.

In [4]:
# разделим данные на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split

X = data.drop('TARGET', axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)

В задании по методу ближайших соседей было найдено, что оптимальное число соседей k = 9.

Обучите на тренировочных данных KNN с k = 9 и выведите матрицу ошибок, а также значение метрик precision и recall на тестовых данных.

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

In [7]:
# ваш код здесь
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(Xtrain, ytrain)

pred_test_knn = knn.predict(Xtest)

print('\n', confusion_matrix(ytest, pred_test_knn))
print('precision_score:', precision_score(ytest, pred_test_knn))
print('recall_score:', recall_score(ytest, pred_test_knn))


 [[3982   26]
 [ 553    6]]
precision_score: 0.1875
recall_score: 0.01073345259391771


Какой вывод можно сделать:
- для класса 0 — клиент не откликнулся — мы получили достаточно высокие значения TP в том числе потому, что представителей этого класса больше;
- для класса 1 — клиент откликнулся — мы получили низкие значения TN.

Поэтому значения precision и recall низкие. Модель даёт неудовлетворительные результаты, так как находит мало клиентов, которые откликнутся на предложение.



Обучите логистическую регрессию с параметрами по умолчанию и посмотрите на метрики.

Везде дальше при оценке метрик надо выводить confusion_matrix, precision и recall.

In [8]:
from sklearn.linear_model import LogisticRegression

# ваш код здесь
lr_1 = LogisticRegression()
lr_1.fit(Xtrain, ytrain)

pred_test_lr1 = lr_1.predict(Xtest)

print('\n', confusion_matrix(ytest, pred_test_lr1))
print('precision_score:', precision_score(ytest, pred_test_lr1))
print('recall_score:', recall_score(ytest, pred_test_lr1))


 [[4008    0]
 [ 559    0]]
precision_score: 0.0
recall_score: 0.0


Наша цель — найти как можно больше клиентов, которые откликнутся на предложение. А модель таких не находит. 

Мы помним, что метод predict_proba у логистической регрессии предсказывает математические (то есть корректные) вероятности классов. Предскажите вероятности классов с помощью обученной логистической регрессии на тестовых данных и выведите вероятности положительного класса для первых десяти объектов. 

Глядя на полученные вероятности, попробуйте объяснить, почему вы получили именно такую матрицу ошибок и такие значения точности с полноты.

In [13]:
# ваш код здесь
probe_test_lr1 = lr_1.predict_proba(Xtest)
print(probe_test_lr1[:10, 1])

[0.14042412 0.14107445 0.19378158 0.27151564 0.12612216 0.04103151
 0.04261878 0.03656178 0.08229851 0.06786435]


In [ ]:
# ваше объяснение здесь

# вероятность класса 1 низкая, т.е. значительно ниже порога 0,5

Давайте уточним цель. Пусть нам нужно найти как можно больше клиентов, которые откликнутся на предложение, то есть максимизировать полноту (recall). 

При этом хочется, чтобы точность модели (precision) не была очень низкой. Заказчик посмотрел на результаты работы KNN и потребовал, чтобы точность была не ниже, чем у KNN: $precision \geq 0.13$.

Давайте будем изменять порог для перевода вероятности в классы так, чтобы:
*   максимизировать значение recall
*   при условии, что $precision \geq 0.13$.

Если мы будем подбирать порог по тестовой выборке, то, по сути, обучимся на ней и, значит, переобучимся. Это плохо. 

Поэтому предскажите вероятности на валидационной выборке и подберите порог по ней (Xval, yval), а затем посмотрите, какое качество для найденного порога вы получите на тестовых данных.

In [18]:
probs_val = lr_1.predict_proba(Xval)[:, 1]

max_recall = -1
thr = -1
prec = -1

for threshold in np.arange(0.05, 1, 0.001):
    # для каждого значения порога переведите вероятности в классы
    # посчитайте метрики
    pred_val = (probs_val > threshold).astype(int)
    
    precision = precision_score(yval, pred_val)
    recall = recall_score(yval, pred_val)
    
    if (recall > max_recall) and (precision >= 0.13):
        max_recall = recall
        thr = threshold
        prec = precision

print('max_recall', max_recall)
print('prec', prec)
print('thr', thr)
# напечатайте порог, для которого получается максимальная полнота, при precision >= 0.13

max_recall 0.8696883852691218
prec 0.1310285958173282
thr 0.07300000000000002


In [19]:
# по найденному порогу переведите вероятности в классы на тесте и напечатайте метрики
probs_test = lr_1.predict_proba(Xtest)[:, 1]
pred_test = (probs_test > thr).astype(int)

print('\n', confusion_matrix(ytest, pred_test))
print('precision_score:', precision_score(ytest, pred_test))
print('recall_score:', recall_score(ytest, pred_test))


 [[1152 2856]
 [ 104  455]]
precision_score: 0.13742071881606766
recall_score: 0.813953488372093


Сделайте вывод. Смогли ли мы с помощью подбора порога добиться большего значения recall, чем у KNN? 

In [ ]:
# ваш вывод здесь
# Да, recall значительно увеличился

А ещё, чтобы улучшить качество предсказания, можно подбирать значение гиперпараметра C у логистической регрессии. Для каждого значения C придётся подбирать свой порог, поэтому:  

1. Обучите для значений C из диапазона [0.05, 0.15, 0.25, ...., 10.05] логистическую регрессию (на тренировочных данных).

2. Для каждой из обученных моделей во внутреннем цикле подберите оптимальный порог (как в предыдущем задании) — на валидационных данных.



В качестве результата выведите значение C и порога для модели, которая даёт наилучшие значения метрик (наибольший recall при ограничении на $precision \geq 0.13$).

Также напечатайте полученные метрики (матрицу ошибок, точность и полноту) для лучшей модели — на тестовых данных.

In [23]:
regs = []
recalls = []
thresholds = []
precisions = []

for reg in np.arange(0.05, 10.1, 0.1):

    # обучите логистическую регрессию с C=reg
    lrm = LogisticRegression(C=reg)
    lrm.fit(Xtrain, ytrain)
    lr_probe = lrm.predict_proba(Xval)[:, 1]
    
    max_recall = -1
    thr = -1
    prec = -1

    for threshold in np.arange(0.05, 0.25, 0.001):
        # подберите оптимальный порог как в задании выше
        pred_val = (lr_probe > threshold).astype(int)
        
        precision = precision_score(yval, pred_val)
        recall = recall_score(yval, pred_val)

        if (recall > max_recall) and (precision >= 0.13):
            max_recall = recall
            thr = threshold
            prec = precision

    recalls.append(max_recall)
    thresholds.append(thr)
    precisions.append(prec)
    regs.append(reg)

In [41]:
# выведите значения C, precision, recall, threshold для наилучшей по заданным условиям модели
params = pd.DataFrame(data=np.vstack((recalls, thresholds, precisions, regs)).T, 
                      columns=['recalls', 'thresholds', 'precisions', 'regs'])
params

,recalls,thresholds,precisions,regs
0,0.869688,0.073,0.131029,0.05
1,0.869688,0.073,0.131029,0.15
2,0.869688,0.073,0.131029,0.25
3,0.869688,0.073,0.131029,0.35
4,0.869688,0.073,0.131029,0.45
...,...,...,...,...
96,0.869688,0.073,0.131029,9.65
97,0.869688,0.073,0.131029,9.75
98,0.869688,0.073,0.131029,9.85
99,0.869688,0.073,0.131029,9.95


In [42]:
params[['recalls', 'thresholds', 'precisions']].value_counts()

recalls   thresholds  precisions
0.869688  0.073       0.131029      101
Name: count, dtype: int64

In [43]:
# с помощью найденных C и threshold обучите модель на тренировочных данных, 
# сделайте предсказание на тесте и по найденному порогу получите классы
# напечатайте метрики

lrm = LogisticRegression()
lrm.fit(Xtrain, ytrain)

lr_probe = lrm.predict_proba(Xtest)[:, 1]
pred_test = (lr_probe > thr).astype(int)

precision = precision_score(ytest, pred_test)
recall = recall_score(ytest, pred_test)
        
print('\n', confusion_matrix(ytest, pred_test))
print('precision_score:', precision)
print('recall_score:', recall)


 [[1152 2856]
 [ 104  455]]
precision_score: 0.13742071881606766
recall_score: 0.813953488372093


Влияет ли изменение гиперпараметра C на качество модели (и, соответственно, метрики) в этой задаче?

In [ ]:
# ваш вывод здесь
# Гиперпараметр С никак не повлиял на качество модели и на метрики

Ответьте развёрнуто на следующие вопросы:

* Удалось ли при помощи логистической регрессии и подбора порога превзойти качество метода ближайших соседей в этой задаче? 

* Смогли ли мы при помощи этой модели получить высокий recall при ограничениях, поставленных заказчиком?


In [ ]:
# ваш вывод здесь

Методом ближайших соседей были получены низкие значения точности и полноты:
precision_score: 0.1875
recall_score: 0.0107

Предсказания модели логистической регрессии дали нулевой результат, что является следствием несбалансированности датасета: количество положительных классов меньше количества отрицательных. 

Снижение порога для выбора положительного класса позволило значительно повысить полноту модели логистической регрессии при ограничениях заданных на точность модели (𝑝𝑟𝑒𝑐𝑖𝑠𝑖𝑜𝑛≥0.13).

Изменение параметра регуляризации не повлияло на качество модели.


